In [1]:
import math, string
import numpy as np
import pandas as pd
import pymysql
import pymysql.cursors as cursors
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchmetrics.functional.classification import f1_score

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator


In [2]:
conn = pymysql.connect(
    host="1.251.203.204",
    port=33065,
    user="root",
    password="kdt5",
    db="Team4",
    charset="utf8",
)
cur = conn.cursor(cursors.DictCursor)


In [3]:
sql = """
select en.text as en, fr.text as fr
from language_en as en
inner join language_fr as fr
on en.id = fr.id
"""


In [4]:
cur.execute(sql)
langDF_ori = pd.DataFrame(cur.fetchall())
cur.close()
conn.close()


In [5]:
punc = string.punctuation.replace("'", "").replace("`", "")
punc


'!"#$%&()*+,-./:;<=>?@[\\]^_{|}~'

In [6]:
# langDF = langDF_ori.replace(f'[{punc}]', '', regex=True).applymap(lambda x: x.lower())
langDF = langDF_ori.applymap(lambda x: x.lower())
langDF


,en,fr
0,in the beginning god created the heaven and th...,"au commencement, dieu créa les cieux et la terre."
1,"and the earth was without form, and void; and ...",la terre était informe et vide: il y avait des...
2,"and god said, let there be light: and there wa...",dieu dit: que la lumière soit! et la lumière fut.
3,"and god saw the light, that it was good: and g...",dieu vit que la lumière était bonne; et dieu s...
4,"and god called the light day, and the darkness...","dieu appela la lumière jour, et il appela les ..."
...,...,...
31097,"and the spirit and the bride say, come. and le...",et l`esprit et l`épouse disent: viens. et que ...
31098,for i testify unto every man that heareth the ...,je le déclare à quiconque entend les paroles d...
31099,and if any man shall take away from the words ...,et si quelqu`un retranche quelque chose des pa...
31100,"he which testifieth these things saith, surely...","celui qui atteste ces choses dit: oui, je vien..."


In [7]:
def generate_tokens(text_iter, language):
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
    for text in text_iter:
        yield token_transform[language](text[language_index[language]])


SRC_LANGUAGE = "en"
TGT_LANGUAGE = "fr"
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ["<unk>", "<pad>", "<bos>", "<eos>"]


token_transform = {
    SRC_LANGUAGE: get_tokenizer("spacy", language="en_core_web_sm"),
    TGT_LANGUAGE: get_tokenizer("spacy", language="fr_core_news_sm"),
}

train_iter = langDF.values

len_en = pd.Series([len(token) for token in generate_tokens(train_iter, SRC_LANGUAGE)])
len_fr = pd.Series([len(token) for token in generate_tokens(train_iter, TGT_LANGUAGE)])
len_en.describe(), len_fr.describe()


(count    31102.000000
 mean        29.475789
 std         12.630380
 min          3.000000
 25%         20.000000
 50%         27.000000
 75%         37.000000
 max        103.000000
 dtype: float64,
 count    31102.000000
 mean        27.692013
 std         11.660609
 min          3.000000
 25%         19.000000
 50%         26.000000
 75%         35.000000
 max         96.000000
 dtype: float64)

In [8]:
def generate_tokens(text_iter, language):
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
    for text in text_iter:
        yield token_transform[language](text[language_index[language]])


SRC_LANGUAGE = "en"
TGT_LANGUAGE = "fr"
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ["<unk>", "<pad>", "<bos>", "<eos>"]


token_transform = {
    SRC_LANGUAGE: get_tokenizer("spacy", language="en_core_web_sm"),
    TGT_LANGUAGE: get_tokenizer("spacy", language="fr_core_news_sm"),
}

vocab_transform = {}
for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    train_iter = langDF.values
    vocab_transform[language] = build_vocab_from_iterator(
        generate_tokens(train_iter, language),
        min_freq=1,
        specials=special_symbols,
        special_first=True,
    )

for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[language].set_default_index(UNK_IDX)

print(vocab_transform)
print(len(vocab_transform["en"]))
print(len(vocab_transform["fr"]))


{'en': Vocab(), 'fr': Vocab()}
12518
24646


In [9]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )

        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)


class TokenEmbedder(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


class Seq2SeqTransformer(nn.Module):
    def __init__(
        self,
        num_encoder_layers,
        num_decoder_layers,
        emb_size,
        max_len,
        nhead,
        src_vocab_size,
        tgt_vocab_size,
        dim_feedforward,
        dropout=0.1,
    ):
        super().__init__()
        self.src_tok_emb = TokenEmbedder(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedder(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, max_len, dropout)
        self.transformer = nn.Transformer(
            d_model=emb_size,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.generator = nn.Linear(emb_size, tgt_vocab_size)

    def forward(
        self,
        src,
        trg,
        src_mask,
        tgt_mask,
        src_padding_mask,
        tgt_padding_mask,
        memory_key_padding_mask,
    ):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(
            src=src_emb,
            tgt=tgt_emb,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            memory_mask=None,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=memory_key_padding_mask,
        )
        return self.generator(outs)

    def encode(self, src, src_mask):
        return self.transformer.encoder(
            self.positional_encoding(self.src_tok_emb(src)), src_mask
        )

    def decode(self, tgt, memory, tgt_mask):
        return self.transformer.decoder(
            self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask
        )


In [10]:
from torchinfo import summary

BATCH_SIZE = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = Seq2SeqTransformer(
    num_encoder_layers=3,
    num_decoder_layers=3,
    emb_size=256,
    max_len=128,
    nhead=8,
    src_vocab_size=len(vocab_transform[SRC_LANGUAGE]),
    tgt_vocab_size=len(vocab_transform[TGT_LANGUAGE]),
    dim_feedforward=512,
).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
optimizer = optim.Adam(model.parameters())

summary(model)


c:\ProgramData\miniconda3\envs\Torch_NLP\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Layer (type:depth-idx)                                                 Param #
Seq2SeqTransformer                                                     --
├─TokenEmbedder: 1-1                                                   --
│    └─Embedding: 2-1                                                  3,204,608
├─TokenEmbedder: 1-2                                                   --
│    └─Embedding: 2-2                                                  6,309,376
├─PositionalEncoding: 1-3                                              --
│    └─Dropout: 2-3                                                    --
├─Transformer: 1-4                                                     --
│    └─TransformerEncoder: 2-4                                         --
│    │    └─ModuleList: 3-1                                            1,581,312
│    │    └─LayerNorm: 3-2                                             512
│    └─TransformerDecoder: 2-5                                         --
│    │    └

In [11]:
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input

    return func


def input_transform(token_ids):
    return torch.cat(
        (torch.tensor([BOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX])),
    )


def collator(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch


text_transform = {}
for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[language] = sequential_transforms(
        token_transform[language], vocab_transform[language], input_transform
    )

data_iter = langDF.values
dataloader = DataLoader(data_iter, batch_size=BATCH_SIZE, collate_fn=collator)
source_tensor, target_tensor = next(iter(dataloader))

print("(source, target): ")
print(next(iter(data_iter)))

print("source_batch:", source_tensor.shape)
print(source_tensor)

print("target_batch:", target_tensor.shape)
print(target_tensor)


(source, target): 
['in the beginning god created the heaven and the earth.'
 'au commencement, dieu créa les cieux et la terre.']
source_batch: torch.Size([63, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  12,    6,    6,  ...,    6,    6,    6],
        [   5,    5,   34,  ...,   30, 3120, 3120],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
target_batch: torch.Size([65, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  41,    8,   34,  ...,   55, 3632, 3632],
        [ 953,   93,   37,  ...,   10,    4,    4],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])


In [12]:
def generate_square_subsequnt_mask(s):
    mask = (torch.triu(torch.ones((s, s), device=DEVICE)) == 1).transpose(0, 1)
    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.float)
    tgt_mask = generate_square_subsequnt_mask(tgt_seq_len)

    src_padding_mask = (src == PAD_IDX).type(torch.float).transpose(0, 1).to(DEVICE)
    tgt_padding_mask = (tgt == PAD_IDX).type(torch.float).transpose(0, 1).to(DEVICE)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


target_input = target_tensor[:-1, :]
target_out = target_tensor[1:, :]

source_mask, target_mask, src_padding_mask, tgt_padding_mask = create_mask(
    source_tensor, target_input
)

print("source_mask:", source_mask.shape)
print(source_mask)
print("target_mask:", target_mask.shape)
print(target_mask)
print("src_padding_mask:", src_padding_mask.shape)
print(src_padding_mask)
print("tgt_padding_mask:", tgt_padding_mask.shape)
print(tgt_padding_mask)


source_mask: torch.Size([63, 63])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
target_mask: torch.Size([64, 64])
tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
src_padding_mask: torch.Size([128, 63])
tensor([[0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.]], device='cuda:0')
tgt_padding_mask: torch.Size([128, 64])
tensor([[

In [13]:
from tqdm import tqdm


def run(epochs, epoch, model, optimizer, criterion, is_train, use_pbar=True):
    model.train() if is_train else model.eval()
    data_iter = langDF.values
    dataloader = DataLoader(data_iter, batch_size=BATCH_SIZE, collate_fn=collator)
    pbar = tqdm(dataloader, total=len(list(dataloader))) if use_pbar else dataloader
    losses = 0
    for source_batch, target_batch in pbar:
        source_batch = source_batch.to(DEVICE)
        target_batch = target_batch.to(DEVICE)

        target_input = target_batch[:-1, :]
        target_output = target_batch[1:, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(
            source_batch, target_input
        )

        logits = model(
            src=source_batch,
            trg=target_input,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            src_padding_mask=src_padding_mask,
            tgt_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=src_padding_mask,
        )

        optimizer.zero_grad()
        loss = criterion(
            logits.reshape(-1, logits.shape[-1]), target_output.reshape(-1)
        )
        if is_train:
            loss.backward()
            optimizer.step()
            if use_pbar:
                pbar.set_description(
                    f"Epoch: {epoch}/{epochs} | Loss: {loss.item():.6f}"
                )
        losses += loss.item()
    return losses / len(list(dataloader))


In [39]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

scheduler = ReduceLROnPlateau(optimizer, mode="min", patience=10, verbose=True)


c:\ProgramData\miniconda3\envs\Torch_NLP\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [31]:
EPOCHS = 100
pre_loss = 0.6


In [40]:
model = torch.load("./bible_transformer_2.pth", map_location=DEVICE)


In [41]:
for epoch in range(1, EPOCHS + 1):
    loss = run(EPOCHS, epoch, model, optimizer, criterion, is_train=True, use_pbar=True)
    scheduler.step(loss)
    torch.save(model, f"bible_transformer_3.pth")
    torch.save(model.state_dict(), f"bible_transformer_state_3.pth")
    print(f"Model Saved! --- Epoch: {epoch}, Loss: {loss:.6f}")


Epoch: 1/100 | Loss: 0.759424: 100%|██████████| 243/243 [00:17<00:00, 13.91it/s]


Model Saved! --- Epoch: 1, Loss: 1.092651


Epoch: 2/100 | Loss: 0.771764: 100%|██████████| 243/243 [00:17<00:00, 13.90it/s]


Model Saved! --- Epoch: 2, Loss: 1.095540


Epoch: 3/100 | Loss: 0.739536: 100%|██████████| 243/243 [00:17<00:00, 13.92it/s]


...

Model Saved! --- Epoch: 98, Loss: 1.093935


Epoch: 99/100 | Loss: 0.739760: 100%|██████████| 243/243 [00:17<00:00, 13.87it/s]


Model Saved! --- Epoch: 99, Loss: 1.093706


Epoch: 100/100 | Loss: 0.757154: 100%|██████████| 243/243 [00:17<00:00, 13.69it/s]


Model Saved! --- Epoch: 100, Loss: 1.093701


In [23]:
def greedy_decode(model, source_tensor, source_mask, max_len, start_symbol):
    source_tensor = source_tensor.to(DEVICE)
    source_mask = source_mask.to(DEVICE)

    memory = model.encode(source_tensor, source_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len - 1):
        memory = memory.to(DEVICE)
        target_mask = generate_square_subsequnt_mask(ys.size(0)).to(DEVICE)
        target_mask = target_mask.type(torch.float).to(DEVICE)

        out = model.decode(ys, memory, target_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat(
            [ys, torch.ones(1, 1).type_as(source_tensor.data).fill_(next_word)], dim=0
        )
        if next_word == EOS_IDX:
            break
    return ys


def translate(model, source_sentence):
    model.eval()
    source_tensor = text_transform[SRC_LANGUAGE](source_sentence.lower()).view(-1, 1)
    num_tokens = source_tensor.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.float).to(DEVICE)
    tgt_tokens = greedy_decode(
        model, source_tensor, src_mask, max_len=num_tokens + 10, start_symbol=BOS_IDX
    ).flatten()
    output = vocab_transform[TGT_LANGUAGE].lookup_tokens(
        list(tgt_tokens.cpu().numpy())
    )[1:-1]
    return " ".join(output)


In [36]:
pre_model = torch.load("bible_transformer_2.pth", map_location=DEVICE)


In [37]:
output1 = translate(pre_model, "In the beginning God created the heaven and the earth.")
output2 = translate(
    pre_model,
    "And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.",
)
output3 = translate(pre_model, "And God said, Let there be light: and there was light.")
print(output1)
print(output2)
print(output3)


et dieu a choisi créature devant jésus christ notre seigneur , et l`appelle seigneur , dont nous croyons en lui disant
et j`entendis du ciel une voix forte voix qui disait : lève - toi , sors promptement de l`arche , pour dieu , et cet homme , pour dieu de dieu , qui soit ressuscité des morts .
dieu dit à noé : que ce songe , fils de dieu a fait cela est à tous les cieux ,


In [42]:
output1_1 = translate(model, "In the beginning God created the heaven and the earth.")
output2_1 = translate(
    model,
    "And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.",
)
output3_1 = translate(model, "And God said, Let there be light: and there was light.")
print(output1_1)
print(output2_1)
print(output3_1)


et dieu a choisi créature devant jésus christ notre seigneur , et l`appelle seigneur , dont nous croyons en lui disant
et j`entendis du ciel une voix forte voix qui disait : lève - toi , sors promptement de l`arche , pour dieu , et cet homme , pour dieu de dieu , qui soit ressuscité des morts .
dieu dit à noé : que ce songe , fils de dieu a fait cela est à tous les cieux ,


In [22]:
import pickle

with open("token_transform.pkl", "wb") as f:
    pickle.dump(token_transform, f)
with open("vocab_transform.pkl", "wb") as f:
    pickle.dump(vocab_transform, f)
